In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import sys
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

In [ ]:
par_dir = os.getcwd() #Parent Directory
par_dir = Path(par_dir)
sys.path.append(str(par_dir))
#Change directory to parent directory if necessary
if os.getcwd() == '/home/USACE_Modeling':
    None
else:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [ ]:
#Basic Data
comm_code = pd.read_csv(par_dir / 'Data' / 'WCSC_Commodity_CodesList.csv')
#Keep unique SingleDigitCode and SingleDigitDescription from comm_code
comm_code = comm_code[['SingleDigitCode','SingleDigitDescription']].drop_duplicates()
#Rename SingleDigitCode to Commodity, SingleDigitDescription to Commodity_Description
comm_code.rename(columns={'SingleDigitCode':'Commodity', 'SingleDigitDescription':'Commodity_Description'}, inplace=True)

wcs_ais_df = pd.read_csv(par_dir / 'Data' / 'UARK_WCS_AIS_Compiled_NewData.csv')
terminal_type = wcs_ais_df[['sub_folder','dominant_vessel_type']].drop_duplicates()

psa = pd.read_csv(par_dir / 'Data' / 'location_with_consistent_data_newdata.csv')
#Remove sub_folder from psa that are not in wcs_ais_df
psa = psa[psa['sub_folder'].isin(wcs_ais_df['sub_folder'])]

actuals = pd.read_csv(par_dir / 'Data' / 'UARK_WCS_AIS_Compiled_NewData_No_Aggregation.csv')
#Keep qurater, sub_folder and columns starting with C_
actuals = actuals[['quarter', 'sub_folder'] + [col for col in actuals.columns if col.startswith('C_')]]
#Melt actuals
actuals = pd.melt(actuals, id_vars=['quarter', 'sub_folder'], var_name='Commodity', value_name='Actuals')
#Rename sub_folder to Terminal
actuals.rename(columns={'sub_folder':'Terminal'}, inplace=True)
#Keep only last digit of Commodity
actuals['Commodity'] = actuals['Commodity'].str[-1]
#Remove records where quarter are in [2021Q1, 2021Q2, 2021Q3, 2021Q4]
actuals = actuals[~actuals['quarter'].isin(['2021Q1', '2021Q2', '2021Q3', '2021Q4'])]
#Convert Commodity to int
actuals['Commodity'] = actuals['Commodity'].astype(int)
#Merge actuals with psa on Terminal on left, sub_folder on right
actuals = pd.merge(actuals,psa, left_on='Terminal', right_on='sub_folder', how='left')
#Drop sub_folder
actuals.drop(columns=['sub_folder'], inplace=True)
#Rename folder to PSA
actuals.rename(columns={'folder':'PSA'}, inplace=True)
actuals.head()
#Export as commodityvolume.csv
# actuals.to_csv('commodityvolume.csv', index=False)

In [ ]:
terminal_type[terminal_type['sub_folder']=='CC_PortOfCC_Southside Terminal']

In [ ]:
LSTM_folder = 'Result_3_1'
TFT_folder = 'Result_3'
TCN_folder = 'Result_3'

In [ ]:
#Data
lstm = pd.read_csv(par_dir / 'Outputs' / 'LSTM_Outputs' / LSTM_folder / 'LSTM_Predictions.csv')
tft = pd.read_csv(par_dir / 'Outputs' / 'TFT_Outputs' / TFT_folder / 'TFT_Predictions.csv')
tcn = pd.read_csv(par_dir / 'Outputs' / 'TCN_Outputs' / TCN_folder / 'TCN_Predictions.csv')

In [ ]:
#Organize columns to [quarter, sub_folder, Actuals, Predictions, Model, Aggregation, Set, Commodity, folder
tft = tft[['quarter', 'sub_folder', 'Actuals', 'Predictions', 'Commodity', 'Model', 'Aggregation']]
lstm = lstm[['quarter', 'sub_folder', 'Actuals', 'Predictions', 'Commodity', 'Model', 'Aggregation']]
tcn = tcn[['quarter', 'sub_folder', 'Actuals', 'Predictions', 'Commodity', 'Model', 'Aggregation']]
#Rename Model to Model
#Vertically stack the dataframes
all_models = pd.concat([lstm,tft, tcn], axis=0)
#Keep last digit of commodity column
all_models['commodity'] = all_models['Commodity'].str[-1]
#Convert Commodity to int
all_models['Commodity'] = all_models['commodity'].astype(int)
#Merge with comm_code to get commodity description
all_models = pd.merge(all_models, comm_code, on='Commodity', how='left')
#Drop Commmodity column
# all_models.drop(columns=['Commodity'], inplace=True)
#Rename Commodity to CommodityCode
all_models.rename(columns={'Commodity':'CommodityCode'}, inplace=True)
#Rename Commodity_Description to Commodity
all_models.rename(columns={'Commodity_Description':'Commodity'}, inplace=True)
#Merge with terminal_type to get dominant_vessel_type
all_models = pd.merge(all_models, terminal_type, on='sub_folder', how='left')
#Merge with psa to get folder
all_models = pd.merge(all_models, psa, on='sub_folder', how='left')
#Create column with value Not Categorized if Aggregation is 'No_Aggregation' else 'Categorized'
all_models['Terminals_Categorized'] = np.where(all_models['Aggregation']=='No_Aggregation', 'Uncategorized', 'Categorized')
#Reorganize columns to [quarter, sub_folder, Actuals, Predictions, Model, Aggregation, Set, dominant_vessel_type, Commodity, folder]
all_models = all_models[['quarter', 'sub_folder', 'Actuals', 'Predictions', 'Model', 'folder', 'Aggregation', 'Terminals_Categorized' , 'dominant_vessel_type', 'CommodityCode', 'Commodity']]

In [ ]:
len(all_models)

In [ ]:
#Remove outliers
outlier_terminals = pd.read_csv(par_dir / 'Data' / 'outlier_terminals.csv')
outlier_terminals_commodity = pd.read_csv(par_dir / 'Data' / 'outlier_terminals_commodity.csv')
#Remove records from outlier terminals whose sub_folder are in outlier_terminal['terminal']
all_models = all_models[~all_models['sub_folder'].isin(outlier_terminals['terminal'])]
#Remove records from all_models whose sub_folder is in outlier_terminals_commodity['terminal'] and CommodityCode is in outlier_terminals_commodity['commodity']
all_models = all_models[~((all_models['sub_folder'].isin(outlier_terminals_commodity['terminal'])) & (all_models['CommodityCode'].isin(outlier_terminals_commodity['commodity'])))]
#Rename sub_folder to Terminal, folder to PSA, dominant_vessel_type to Terminal Type, Terminals_Categorized to Categorization
all_models.rename(columns={'sub_folder':'Terminal', 'folder':'PSA', 'dominant_vessel_type':'Terminal Type', 'Terminals_Categorized':'Categorization'}, inplace=True)

#Change negative predictions to 0
all_models['Predictions'] = np.where(all_models['Predictions']<0, 0, all_models['Predictions'])

In [ ]:
#Show records where Predictions are zero and Model = LSTM and Aggregation = No_Aggregation
all_models[(all_models['Predictions']<0) & (all_models['Model']=='LSTM') & (all_models['Aggregation']=='No_Aggregation')]

## MAPE - Testing

In [ ]:
#MAE by Model
mape_df = all_models.copy()
mape_df['error'] = abs(mape_df['Actuals'] - mape_df['Predictions'])
#Sum error and Actuals by Model, Aggregation, Commodity
mape_df = mape_df.groupby(['Model', 'CommodityCode', 'Terminal', 'PSA', 'Terminal Type', 'Categorization', 'Commodity'], as_index=False).agg({'error':'sum', 'Actuals':'sum'})
mape_df['MAPE'] = mape_df['error']/mape_df['Actuals']*100
mape_df.head()

### Plots

In [ ]:
#Global par#Settings for plotly graphs
GLOBAL_LAYOUT = dict(
    #Theme to pltoly_white
    template='plotly_white',
    #Title at center
    title_x=0.5,
    #Font size
    font=dict(size=30),
    #Black x and y axis lines
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    #No gridlines
    xaxis_showgrid=False,
    yaxis_showgrid=False,
)
#Discrete colors for Terminal Type
c_terminal = dict(zip(mape_df['Terminal Type'].unique(), px.colors.qualitative.Safe))
#Discrete patterns for Terminal Type
p_terminal = dict(zip(mape_df['Terminal Type'].unique(), ['.', '/', '+','\\']))

In [ ]:
#Plot for MAPE by Model and Terminal Categorization
df = mape_df.copy()
#Average mape by folder
df = df.groupby(['Model', 'Categorization'], as_index=False).agg({'MAPE':'mean'})
#Get value of Model and Categorization for lowest MAPE
best_model = df[df['MAPE']==df['MAPE'].min()][['Model', 'Categorization']].values[0][0]
best_category = df[df['MAPE']==df['MAPE'].min()][['Model', 'Categorization']].values[0][1]

#Plot df
fig = px.bar(df, x='Model', y='MAPE', color='Categorization', barmode='group', title='MAPE by Model and Terminal Categorization')
#Rename y axis label to MAPE (%)
fig.update_yaxes(title_text='MAPE (%)')
#Show mape on the top center of each bar
fig.update_traces(texttemplate='%{y:.1f}'+'%', textposition='outside')
#Update layout
fig.update_layout(GLOBAL_LAYOUT)
fig.show()
df.head()

In [ ]:
#Plot for MAPE by Model for Uncategorized
df = mape_df.copy()
#Keep records where Categorization = Uncategorized
# df = df[df['Categorization']=='Uncategorized']
#Average mape by folder
df = df.groupby(['Model', 'Categorization'], as_index=False).agg({'MAPE':'mean'})
#Plot df
fig = px.bar(df, x='Model', y='MAPE', color='Categorization',title='MAPE by Model', barmode='group')
#Rename y axis label to MAPE (%)
fig.update_yaxes(title_text='MAPE (%)')
#Show mape on the top center of each bar
fig.update_traces(texttemplate='%{y:.1f}'+'%', textposition='outside')
#Update layout
fig.update_layout(GLOBAL_LAYOUT)
fig.update_layout(legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.01))
fig.show()

In [ ]:
df = mape_df[(mape_df['Categorization']=='Uncategorized')]
#Average mape by folder
df = df.groupby(['Model','Terminal'], as_index=False).agg({'MAPE':'mean'})

In [ ]:
terminal_example = 'tampa_tanker_manatee_county'
commodity_example = 2
# df[df['Terminal']==terminal_example]

In [ ]:
tft_lstm_compare = all_models[(all_models['Terminal']==terminal_example) & (all_models['Categorization']=='Uncategorized')]
#Keep records from tft_lstm_compare whose Terminal is terminal_example and commodity is commodity_example
tft_lstm_compare = tft_lstm_compare[(tft_lstm_compare['Terminal']==terminal_example) & (tft_lstm_compare['CommodityCode']==commodity_example)]
tft_lstm_compare.head()

In [ ]:
#Create line plot of actuals vs quarter from actuals dataframe for Terminal = terminal_example and Commodity = commodity_example
actuals_plot = actuals[(actuals['Terminal']==terminal_example)&(actuals['Commodity']==commodity_example)].copy()
fig = px.line(actuals_plot, x='quarter', y='Actuals')
#Add horizontal average line for training quarters from 2016Q1 to 2018Q4
fig.add_hline(y=actuals_plot[(actuals_plot['quarter']>='2016Q1') & (actuals_plot['quarter']<='2018Q4')]['Actuals'].mean(), line_dash="dash", line_color="green", annotation_text="Average of Training Set", annotation_position="top left")

# Add actuals line with a legend
fig.add_trace(go.Scatter(x=actuals_plot['quarter'], y=actuals_plot['Actuals'], mode='lines', name='Actuals'))
#Add Predictions from tft_lstm_compare to fig where Model= LSTM
fig.add_scatter(x=tft_lstm_compare[tft_lstm_compare['Model']=='LSTM']['quarter'], y=tft_lstm_compare[tft_lstm_compare['Model']=='LSTM']['Predictions'], mode='lines', name='LSTM')
#Add Predictions from tft_lstm_compare to fig where Model= TFT
fig.add_scatter(x=tft_lstm_compare[tft_lstm_compare['Model']=='TFT']['quarter'], y=tft_lstm_compare[tft_lstm_compare['Model']=='TFT']['Predictions'], mode='lines', name='TFT')
#Add Predictions from tft_lstm_compare to fig where Model= TCN
fig.add_scatter(x=tft_lstm_compare[tft_lstm_compare['Model']=='TCN']['quarter'], y=tft_lstm_compare[tft_lstm_compare['Model']=='TCN']['Predictions'], mode='lines', name='TCN')
#Add Title
fig.update_layout(title='Actuals vs Predictions for '+terminal_example+' and Commodity = Petroleum')
#Rename y axis label to Tons
fig.update_yaxes(title_text='Tons')
fig.update_layout(GLOBAL_LAYOUT)
#Show legend
fig.show()

### Plots for Best Model

In [ ]:
px.colors.qualitative.Safe

In [ ]:
#Plot of number of terminals by Terminal Type
df = mape_df.copy()
df = df[['Terminal','PSA','Terminal Type']].drop_duplicates()
#Count number of terminals by Terminal Type
df = df.groupby(['Terminal Type'], as_index=False).agg({'Terminal':'count'})
#Rename Terminal to Number of Terminals
df.rename(columns={'Terminal':'Number of Terminals'}, inplace=True)
#Sort df by Number of Terminals
df.sort_values(by='Number of Terminals', inplace=True)
#Replace Tug_Tow to Tug/Tow
df['Terminal Type'] = np.where(df['Terminal Type']=='Tug_Tow', 'Tug/Tow', df['Terminal Type'])




#Plot df
fig = px.bar(df, 
             x='Terminal Type', 
             y='Number of Terminals', 
             title='Terminal Types Based on Vessel Visits', 
             color='Terminal Type',
             color_discrete_map=c_terminal,
             pattern_shape='Terminal Type',
             pattern_shape_map=p_terminal)
#Change x-axis label to Terminal Type
fig.update_xaxes(title_text='Vessel Type')
#Black font
fig.update_layout(font=dict(color='black'))
#Bold font
fig.update_layout(font=dict(family="Arial, Bold"))

#Remove horizontal grid lines
fig.update_yaxes(showgrid=False)
#Center the title
fig.update_layout(title_x=0.5)
#Black x and y axis
fig.update_xaxes(linecolor='black', mirror=False)
fig.update_yaxes(linecolor='black', mirror=False)
fig.update_traces(texttemplate='%{y:.0f}', textposition='outside')
fig.update_layout(GLOBAL_LAYOUT)
#Hide legend
fig.update_layout(coloraxis_showscale=False)
#25 font size
fig.update_layout(font=dict(size=38))
#Hide lengend
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#Model for figures
category_name = 'Uncategorized'
fig_model_df = mape_df[(mape_df['Categorization']==category_name)]
fig_model_df

In [ ]:
#Sort fig_model_df by MAPE
df = fig_model_df.copy()
df = df[df['Model']=='LSTM']
df.sort_values(by='MAPE', ascending=False)

In [ ]:
actuals[(actuals['Terminal']=='Wilmington_NCSPABerths1_9') & (actuals['Commodity']==6)]

In [ ]:
#By Terminal Type
df = fig_model_df.copy()
df = df[df['Model']=='LSTM']
#Average mape by Terminal Type
df = df.groupby(['Terminal Type', 'Model'], as_index=False).agg({'MAPE':'mean'})
#Sort df by MAPE
df.sort_values(by='MAPE', inplace=True)
#Change Tug_Tow to Tug/Tow
df['Terminal Type'] = np.where(df['Terminal Type']=='Tug_Tow', 'Tug/Tow', df['Terminal Type'])
#Plot df
fig = px.bar(df, x='Terminal Type', y='MAPE', title='MAPE by Terminal Type', color='Terminal Type', 
             color_discrete_map=c_terminal,
             pattern_shape='Terminal Type',
                pattern_shape_map=p_terminal)
#Add black border to bars
# fig.update_traces(marker_line_color='black', marker_line_width=1.5, opacity=0.8)
#Rename y axis label to MAPE (%)
fig.update_yaxes(title_text='MAPE (%)')
fig.update_traces(texttemplate='%{y:.1f}'+'%', textposition='outside')
fig.update_layout(GLOBAL_LAYOUT)
#Legend
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#Plot of MAPE by PSA and Average Volume
df = fig_model_df.copy()
#Average mape by PSA
df = df.groupby(['PSA'], as_index=False).agg({'MAPE':'mean'})

#Average Actuals by PSA
#Sum Actuals by PSA and quarter
actuals_psa_avg = actuals.groupby(['PSA', 'quarter'], as_index=False).agg({'Actuals':'sum'})
#Average of Actuals by PSA
actuals_psa_avg = actuals_psa_avg.groupby(['PSA'], as_index=False).agg({'Actuals':'mean'})
#Rename Actuals to Average Volume
actuals_psa_avg.rename(columns={'Actuals':'Average Volume'}, inplace=True)

#Sort df in the same order as actuals_psa_avg
df = pd.merge(df, actuals_psa_avg[['PSA', 'Average Volume']], on='PSA')
#Sort by Average Volume
df.sort_values(by='Average Volume', inplace=True)

#Plot df as line plot with PSA as x-axis and y-axis as MAPE
fig = px.line(df, x='PSA', y='MAPE', title='MAPE by PSA')
fig.add_trace(go.Scatter(x=df['PSA'], y=df['MAPE'], mode='lines+markers', name='MAPE'))
fig.update_yaxes(title_text='MAPE (%)')
fig.add_trace(go.Bar(x=df['PSA'], y=df['Average Volume'], name='Average Volume', yaxis='y2', opacity=0.8))
# Update layout to add secondary y-axis
fig.update_layout(
    yaxis2=dict(
        title='Tons',
        overlaying='y',
        side='right'
    )
)
#Show legend in bottom right
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(GLOBAL_LAYOUT)
fig.show()

In [ ]:
#Plot by Commodity
df =fig_model_df.copy()
#Average mape by Commodity
df = df.groupby(['Model','Commodity'], as_index=False).agg({'MAPE':'mean'})
#Sort df by MAPE
df.sort_values(by='MAPE', inplace=True)
#Change All Manufactured Equipment, Machinery, and Products to All Manufactured Products; Chemicals and Related Products to Chemicals, Primary Manufactured Goods to Primary Manufactured Goods; Crude Materials, Inedible Except Fuels to Crude Materials; Petroleum and Petroleum Products to Petroleum Products; Coal, Lignite & coal Coke to Coal Products
df['Commodity'] = np.where(df['Commodity']=='All Manufactured Equipment, Machinery and Products', 'All Manufactured Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Chemicals and Related Products', 'Chemicals', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Primary Manufactured Goods', 'Primary Manufactured Goods', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Crude Materials, Inedible Except Fuels', 'Crude Materials', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Petroleum and Petroleum Products', 'Petroleum Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Coal,Lignite & Coal Coke', 'Coal Products', df['Commodity'])

#Sort by Commodity, Model
df.sort_values(by=['Commodity', 'Model'], inplace=True)
#Plot df
fig = px.bar(df, x='Commodity', y='MAPE', title='MAPE by Commodity', color='Model', barmode='group')
#Rename y axis label to MAPE (%)
fig.update_yaxes(title_text='MAPE (%)')
fig.update_traces(texttemplate='%{y:.1f}'+'%', textposition='outside')
#X-axis label at 45 degree angle
fig.update_xaxes(tickangle=0)
fig.update_layout(GLOBAL_LAYOUT)
#Reduce font size of x-axis labels
fig.update_xaxes(tickfont=dict(size=18))
fig.show()

In [ ]:
#Plot by Commodity and Terminal Type
df =fig_model_df.copy()
df = df[df['Model']=='LSTM']
#Average mape by Commodity
df = df.groupby(['Terminal Type','Commodity'], as_index=False).agg({'MAPE':'mean'})
#Sort df by MAPE
df.sort_values(by='MAPE', inplace=True)
#Change All Manufactured Equipment, Machinery, and Products to All Manufactured Products; Chemicals and Related Products to Chemicals, Primary Manufactured Goods to Primary Manufactured Goods; Crude Materials, Inedible Except Fuels to Crude Materials; Petroleum and Petroleum Products to Petroleum Products; Coal, Lignite & coal Coke to Coal Products
df['Commodity'] = np.where(df['Commodity']=='All Manufactured Equipment, Machinery and Products', 'All Manufactured Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Chemicals and Related Products', 'Chemicals', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Primary Manufactured Goods', 'Primary Manufactured Goods', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Crude Materials, Inedible Except Fuels', 'Crude Materials', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Petroleum and Petroleum Products', 'Petroleum Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Coal,Lignite & Coal Coke', 'Coal Products', df['Commodity'])

#Sort by Commodity, Model
df.sort_values(by=['Commodity', 'Terminal Type'], inplace=True)
#Plot df
fig = px.bar(df, x='Commodity', y='MAPE', title='MAPE by Commodity', color='Terminal Type', barmode='group')
#Rename y axis label to MAPE (%)
fig.update_yaxes(title_text='MAPE (%)')
fig.update_traces(texttemplate='%{y:.1f}'+'%', textposition='outside')
#X-axis label at 45 degree angle
fig.update_xaxes(tickangle=0)
fig.update_layout(GLOBAL_LAYOUT)
#Reduce font size of x-axis labels
fig.update_xaxes(tickfont=dict(size=18))
#Hide legend
fig.update_layout(showlegend=True, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

In [ ]:
#Show records from all_model where Predictions=0
zero_predict = all_models[all_models['Predictions']==0]
#Count number of records by Commodity
zero_predict = zero_predict.groupby(['Commodity'], as_index=False).agg({'Predictions':'count'})
#Rename Predictions to Number of Quarters
zero_predict.rename(columns={'Predictions':'Number of Quarters with 0 prediction'}, inplace=True)
#Sort in order
zero_predict.sort_values(by='Number of Quarters with 0 prediction', inplace=True)

#Count Number of records by Commodity
num_records_by_com = all_models.groupby(['Commodity'], as_index=False).agg({'Terminal':'count'})
#Rename Terminal to Total Number of Records
num_records_by_com.rename(columns={'Terminal':'Total Number of Records'}, inplace=True)
#Merge with zero_predict on Commodity
zero_predict = pd.merge(zero_predict, num_records_by_com, on='Commodity', how='left')
#Percentage of records with 0 prediction
zero_predict['Percentage of records with 0 prediction'] = zero_predict['Number of Quarters with 0 prediction']/zero_predict['Total Number of Records']*100
#Sort by Percentage of records with 0 prediction
zero_predict.sort_values(by='Percentage of records with 0 prediction', inplace=True)

#Plot zero_predict as bar plot
fig = px.bar(zero_predict, x='Commodity', y='Percentage of records with 0 prediction', title='Percentage of Records with Prediction = 0 tons')
fig.update_traces(texttemplate='%{y:.1f}', textposition='outside')
fig.update_layout(GLOBAL_LAYOUT)
fig.show()

In [ ]:
# df = best_model_df.copy()
# 
# num_com_by_term = all_models[['Terminal', 'PSA', 'CommodityCode']].drop_duplicates() #Number of commodities handled by each terminal
# #Count number of CommodityCode by Terminal
# num_com_by_term = num_com_by_term.groupby(['Terminal', 'PSA'], as_index=False).agg({'CommodityCode':'count'})
# #Rename CommodityCode to Number of Commodities
# num_com_by_term.rename(columns={'CommodityCode':'Number of Commodities'}, inplace=True)
# 
# #Merge with df on Termina
# df = pd.merge(df, num_com_by_term, on=['Terminal', 'PSA'], how='left')
# 
# #Average mape by Number of Commodities
# df = df.groupby(['Number of Commodities'], as_index=False).agg({'MAPE':'mean'})
# #Sort df by MAPE
# df.sort_values(by='MAPE', inplace=True)
# df

In [ ]:
#Plot of histogram of MAPE by Terminal
df = fig_model_df.copy()
#Average of MAPE by Terminal
df = df.groupby(['Terminal'], as_index=False).agg({'MAPE':'mean'})
#Create bins for MAPE with interval of 10
df['MAPE_bins'] = pd.cut(df['MAPE'], bins=np.arange(0, 2000, 5))
#COnvert MAPE_bins to string
df['MAPE_bins'] = df['MAPE_bins'].astype(str)

#Count number of records by MAPE_bins
df = df.groupby(['MAPE_bins'], as_index=False).agg({'Terminal':'count'})
#Rename Terminal to Number of Terminals
df.rename(columns={'Terminal':'Number of Terminals'}, inplace=True)
#Create Rank columm with value between ( and , in MAPE_bins
df['Rank'] = df['MAPE_bins'].str.split(',').str[0].str[1:]
df['Rank'] = df['Rank'].astype(int)
#Sort df by Rank
df.sort_values(by='Rank', inplace=True)
#Create cumulative sum of Number of Terminals
df['Cumulative Terminals'] = df['Number of Terminals'].cumsum()
#Create cumulative sum of Number of Terminals as percentage of total Number of Terminals
df['Percentage of Terminals'] = df['Cumulative Terminals']/df['Number of Terminals'].sum()*100
# Plot bar plot of df
fig = px.bar(df, x='MAPE_bins', y='Number of Terminals', title='Histogram of MAPE')

# Add labels in the middle of each bin
fig.update_traces(texttemplate='%{y:.0f}', textposition='outside')

# Rename MAPE_bins to MAPE (%)
fig.update_xaxes(title_text='MAPE (%)')
#Rename y axis label to Number of Terminals / Percentage of Terminals
fig.update_yaxes(title_text='Number of Terminals / Percentage of Terminals')

# Add line plot for cumulative percentage of terminals as step plot
fig.add_trace(go.Scatter(
    x=df['MAPE_bins'],
    y=df['Percentage of Terminals'],
    mode='lines+markers',
    name='Cumulative Percentage of Terminals',
    line=dict(shape='hv')
))

# Add labels to the line plot
for i, label in enumerate(df['Percentage of Terminals']):
    fig.add_annotation(
        go.layout.Annotation(
            x=df['MAPE_bins'].iloc[i],
            y=label,
            text=f"{label:.1f}%",
            #Font color to dark red and size to 17
            font=dict(color='darkred', size=17),
            showarrow=True,
            arrowhead=5,
            ax=0,
            ay=-40
        )
    )
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.update_layout(GLOBAL_LAYOUT)
# Show the figure
fig.show()

In [ ]:
#Plot of histogram of MAPE by Terminal and Commodity combination
df = mape_df[(mape_df['Categorization']==category_name)&(mape_df['Model']=='LSTM')].copy()

#Create bins for MAPE with interval of 10
df['MAPE_bins'] = pd.cut(df['MAPE'], bins=np.arange(0, 2000, 5))
#COnvert MAPE_bins to string
df['MAPE_bins'] = df['MAPE_bins'].astype(str)

#Count number of records by MAPE_bins
df = df.groupby(['MAPE_bins'], as_index=False).agg({'Terminal':'count'})
#Rename Terminal to Number of Terminals
df.rename(columns={'Terminal':'Number of Terminals'}, inplace=True)
#Create Rank columm with value between ( and , in MAPE_bins
df['Rank'] = df['MAPE_bins'].str.split(',').str[0].str[1:]
df['Rank'] = df['Rank'].astype(int)
#Sort df by Rank
df.sort_values(by='Rank', inplace=True)
#Create cumulative sum of Number of Terminals
df['Cumulative Terminals'] = df['Number of Terminals'].cumsum()
#Create cumulative sum of Number of Terminals as percentage of total Number of Terminals
df['Percentage of Terminals'] = df['Cumulative Terminals']/df['Number of Terminals'].sum()*100
# Plot bar plot of df
fig = px.bar(df, x='MAPE_bins', y='Number of Terminals')

# Add labels in the middle of each bin
fig.update_traces(texttemplate='%{y:.0f}', textposition='outside')




# Rename MAPE_bins to MAPE (%)
fig.update_xaxes(title_text='MAPE (%)')
#Rename y axis label to Number of Terminals / Percentage of Terminals
fig.update_yaxes(title_text='Number (Percentage) of Terminals-Commodity Combination', tickfont=dict(size=2))

# Add line plot for cumulative percentage of terminals as step plot
fig.add_trace(go.Scatter(
    x=df['MAPE_bins'],
    y=df['Percentage of Terminals'],
    mode='lines+markers',
    name='Cumulative Percentage of Terminals and Commodity',
    line=dict(shape='hv'),
    textfont=dict(size=5)
))

# Add labels to the line plot
for i, label in enumerate(df['Percentage of Terminals']):
    fig.add_annotation(
        go.layout.Annotation(
            x=df['MAPE_bins'].iloc[i],
            y=label,
            text=f"{label:.1f}%",
            #Font color to dark red and size to 17
            font=dict(color='darkred', size=18),
            showarrow=True,
            arrowhead=5,
            ax=0,
            ay=-40
        )
    )
    

fig.update_layout(GLOBAL_LAYOUT)

fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Show the figure
fig.show()
# df.to_csv('Distribution of MAPE for LSTM.csv', index=False)

In [ ]:
#Number of terminals by Commodity group
df = fig_model_df[['Terminal','Commodity', 'Terminal Type']].drop_duplicates(keep='first')
df['Commodity'] = np.where(df['Commodity']=='All Manufactured Equipment, Machinery and Products', 'All Manufactured Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Chemicals and Related Products', 'Chemicals', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Primary Manufactured Goods', 'Primary Manufactured Goods', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Crude Materials, Inedible Except Fuels', 'Crude Materials', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Petroleum and Petroleum Products', 'Petroleum Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Coal,Lignite & Coal Coke', 'Coal Products', df['Commodity'])
df['Terminal Type'] = np.where((df['Terminal Type']=='Tug_Tow'), 'Tug/Tow', df['Terminal Type'])

#Sort by Commodity, Terminal Type
df.sort_values(by=['Commodity', 'Terminal Type'], inplace=True)
#Plot number of terminals by Commodity group and Terminal Type
fig = px.histogram(df, x='Commodity', 
                   color='Terminal Type',
                     color_discrete_map=c_terminal,
                   pattern_shape='Terminal Type',
                     pattern_shape_map=p_terminal,
                   title='Number of Terminals by Commodity Group and Terminal Type',)
#Rename y axis label to Number of Terminals
fig.update_yaxes(title_text='Number of Terminals')
#Rename x axis label to Commodity Group
fig.update_xaxes(title_text='Commodity Group')
fig.update_layout(GLOBAL_LAYOUT)
#Add labels
fig.update_traces(texttemplate='%{y:.0f}', textposition='auto',textfont=dict(size=18))
fig.update_layout(showlegend=True, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

#X axis label at 0 degree with font size 18

fig.show()

In [ ]:
#Lineplot for quarterly volume of comoodity by terminal type
df = actuals.copy()
#Sum Actuals by quarter and Commodity
df = df.groupby(['quarter', 'Commodity'], as_index=False).agg({'Actuals':'sum'})
#Rename Actuals to Quarterly Volume
df.rename(columns = {'Actuals':'Quarterly Volume'}, inplace=True)
#Merge with comm_code to get commodity description
df = pd.merge(df, comm_code, on='Commodity', how='left')
#Drop Commmodity column
df.drop(columns = ['Commodity'], inplace=True)
#Rename Commodity_Description to Commodity
df.rename(columns = {'Commodity_Description':'Commodity'}, inplace=True)

df['Commodity'] = np.where(df['Commodity']=='All Manufactured Equipment, Machinery and Products', 'All Manufactured Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Chemicals and Related Products', 'Chemicals', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Primary Manufactured Goods', 'Primary Manufactured Goods', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Crude Materials, Inedible Except Fuels', 'Crude Materials', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Petroleum and Petroleum Products', 'Petroleum Products', df['Commodity'])
df['Commodity'] = np.where(df['Commodity']=='Coal,Lignite & Coal Coke', 'Coal Products', df['Commodity'])

#Remove records where Commodity starts with Waste or Unknown
df = df[~df['Commodity'].str.startswith('Waste')]
df = df[~df['Commodity'].str.startswith('Unknown')]

#Plot df as line plot with quarter as x-axis and y-axis as Quarterly Volume
fig = px.line(df, x='quarter', y='Quarterly Volume', color='Commodity', title='Quarterly Volume by Commodity', line_dash='Commodity')
#Rename y axis label to Quarterly Volume (Tons)
fig.update_yaxes(title_text='Quarterly Volume (Tons)')
#Rename x axis label to Quarter
fig.update_xaxes(title_text='Quarter')
#Add legend in top right
fig.update_layout(legend=dict(yanchor="top", y=0.70, xanchor="right", x=0.99))
#Add labels
# fig.update_traces(texttemplate='%{y:.0f}', textposition='auto',textfont=dict(size=18))
fig.update_layout(GLOBAL_LAYOUT)
fig.update_xaxes(tickangle=270)
fig.show()
# df.to_csv('Quarterly volume (in tons) by commodity group.csv', index=False)